# Join French Crop Usage (FCU) with TAXREF-LD via GEVES scientific names

Join FCU with TAXREF-LD using GEVES as an intermediate because GEVES has most FCU varieties and provides scientific names.
Process:
- join FCU with GEVES: where FCU variety name = GEVES species name
- join only the matched GEVES species names with the scientific names in TAXREF-LD

### Initializations

In [1]:
import json
import os
from string import Template
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON, POST
from time import sleep
from math import isnan, nan

In [2]:
# Initializations
prefixes = '''
prefix api:            <http://ns.inria.fr/sparql-micro-service/api#>
prefix fcu:            <http://ontology.inrae.fr/frenchcropusage/>
prefix owl:            <http://www.w3.org/2002/07/owl#>
prefix rdf:            <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs:           <http://www.w3.org/2000/01/rdf-schema#>
prefix skos:           <http://www.w3.org/2004/02/skos/core#>
prefix skosxl:         <http://www.w3.org/2008/05/skos-xl#>
prefix taxrefp:        <http://taxref.mnhn.fr/lod/property/>
prefix taxrefrk:       <http://taxref.mnhn.fr/lod/taxrank/>
prefix xsd:            <http://www.w3.org/2001/XMLSchema#>
'''

In [3]:
local_endpoint = 'http://localhost:8080/sparql'
fcu_endpoint = "http://ontology.inrae.fr/frenchcropusage/sparql"
#fcu_endpoint = "http://localhost:8080/sparql"
taxref_endpoint = "https://taxref.mnhn.fr/sparql"
geves_endpoint = "http://graph.i3s.unice.fr/repositories/geves"

In [4]:
def exec_sparql(endpoint, query):
    """
    Execute a SPARQL query and return results as a Pandas DataFrame.
    Invocation uses the HTTP POST method. Requested result format is JSON.

    In case of failure, the function retries up to MAX_ATTEMPTS attempts while waiting increasing time between each attempt.
    An empty DataFrame is returned in case no attempt is successful.
    """
    # Max number of time a SPARQL query can fail before giving up
    MAX_ATTEMPTS = 3

    sparql = SPARQLWrapper(endpoint)
    sparql.setMethod(POST)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    
    attempt = 1;
    success = False
    while not success and attempt <= MAX_ATTEMPTS:
        try:
            raw_results = sparql.query()
            results = raw_results.convert()
        except:
            print(f'Error while executing SPARQL query (attempt {attempt}/{MAX_ATTEMPTS}).', end=' ')
            if attempt < MAX_ATTEMPTS:
                # Wait a few seconds before next attempt (5, 10, 15, ...)
                print(f'Will retry in {attempt * 5}s.')
                sleep(attempt * 5)
        else:
            success = True
        finally:
            attempt = attempt + 1

    if success:
        if 'head' in results and 'results' in results:
            cols = results['head']['vars']
            out = []
            for row in results['results']['bindings']:
                item = []
                for c in cols:
                    item.append(row.get(c, {}).get('value'))
                out.append(item)
            return pd.DataFrame(out, columns=cols)
        else:
            print('Invalid SPARQL result. Will return empty DataFrame.\n' + str(results))
    else:
        print(f'Unable to execute SPARQL query after {MAX_ATTEMPTS} attempts. Will return empty DataFarme.')
                  
    return pd.DataFrame()            


def dataframe_preview(df, start=0, end=10):
    print("== Number of lines: " + str(df.shape[0]))
    print("== Number of unique values:")
    print(df.nunique())
    display(df[start:end])

---
## Join FCU with GEVES: FCU varieties names = GEVES species

In [9]:
query =  prefixes + '''
select distinct ?fcu_concept ?fcu_name ?fcu_name_type ?geves_lab_spe_dus ?geves_scientific_name where {

    # Query FCU crops
    service <''' + fcu_endpoint + '''> {
        select distinct ?fcu_concept ?fcu_name ?fcu_name_type where {

            # Select only cultivated crops
            <http://ontology.inrae.fr/frenchcropusage/Usages_plantes_cultivees> skos:narrower+ ?fcu_concept.

            # Keep only leaves or their direct parent but not above
            FILTER (NOT EXISTS { ?fcu_concept skos:narrower/skos:narrower ?child. })

            # Get prefered and alternate labels and keep track of the type of label
            ?fcu_concept a skos:Concept.
            { ?fcu_concept skos:prefLabel ?lb. bind("pref" as ?fcu_name_type) }
            UNION
            { ?fcu_concept skos:altLabel  ?lb. bind("alt" as ?fcu_name_type) }

            bind(str(lcase(?lb)) as ?fcu_name)
        } order by ?fcu_concept
    }

    # Query GEVES
    optional {
        service <http://graph.i3s.unice.fr/repositories/geves> {
            [] 
                api:lab_spe_bota ?lab_spe_bota;
                api:lab_spe_dus ?geves_lab_spe_dus.
            bind(str(lcase(?lab_spe_bota))      as ?geves_scientific_name)
            bind(str(lcase(?geves_lab_spe_dus)) as ?geves_species)
        }

        # Match the GEVES species names and the FCU preferred/alternate labels:
        filter (?fcu_name = ?geves_species) 
    }
}'''

In [10]:
%time df_fcu_geves = exec_sparql(local_endpoint, query)

Wall time: 12.2 s


In [11]:
dataframe_preview(df_fcu_geves, end=10)

== Number of lines: 1521
== Number of unique values:
fcu_concept               446
fcu_name                 1481
fcu_name_type               2
geves_lab_spe_dus          89
geves_scientific_name      85
dtype: int64


,fcu_concept,fcu_name,fcu_name_type,geves_lab_spe_dus,geves_scientific_name
0,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier,pref,Abricotier,prunus armeniaca l.
1,http://ontology.inrae.fr/frenchcropusage/Abric...,abricot,alt,None,None
2,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier pays,pref,None,None
3,http://ontology.inrae.fr/frenchcropusage/Abric...,abricot pays,alt,None,None
4,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier des antilles,alt,None,None
5,http://ontology.inrae.fr/frenchcropusage/Abric...,mamey,alt,None,None
6,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier-pays,alt,None,None
7,http://ontology.inrae.fr/frenchcropusage/Actin...,actinidia,pref,None,None
8,http://ontology.inrae.fr/frenchcropusage/Actin...,groseille de chine,alt,None,None
9,http://ontology.inrae.fr/frenchcropusage/Actin...,kiwi,alt,None,None


In [12]:
#df_fcu_geves.to_csv("result1_fcu_geves.csv", index=False)
df_fcu_geves.to_excel("result1_fcu_geves.xlsx")
#df_fcu_geves = pd.read_csv("result1_fcu_geves.csv")

#### Count only matches

In [13]:
dataframe_preview(df_fcu_geves.dropna())

== Number of lines: 98
== Number of unique values:
fcu_concept              91
fcu_name                 89
fcu_name_type             2
geves_lab_spe_dus        89
geves_scientific_name    85
dtype: int64


,fcu_concept,fcu_name,fcu_name_type,geves_lab_spe_dus,geves_scientific_name
0,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier,pref,Abricotier,prunus armeniaca l.
13,http://ontology.inrae.fr/frenchcropusage/Ails,ail,pref,Ail,allium sativum l.
21,http://ontology.inrae.fr/frenchcropusage/Amand...,amandier,pref,Amandier,prunus amygdalus bartock
61,http://ontology.inrae.fr/frenchcropusage/Artic...,artichaut,pref,Artichaut,cynara scolymus
62,http://ontology.inrae.fr/frenchcropusage/Artic...,artichaut,pref,Artichaut,cynara cardunculus l.
64,http://ontology.inrae.fr/frenchcropusage/Asperges,asperge,pref,Asperge,asparagus officinalis l.
73,http://ontology.inrae.fr/frenchcropusage/Auber...,aubergine,pref,Aubergine,solanum melongena l.
79,http://ontology.inrae.fr/frenchcropusage/Avoin...,avoine d'hiver,pref,Avoine d'hiver,avena sativa l.
83,http://ontology.inrae.fr/frenchcropusage/Avoin...,avoine nue d'hiver,pref,Avoine nue d'hiver,avena nuda l.
85,http://ontology.inrae.fr/frenchcropusage/Avoin...,avoine nue de printemps,pref,Avoine nue de printemps,avena nuda l.


### Matches:
- FCU: 91 unique concepts, 89 unique labels
- GEVES: 89 unique species, 85 unique scientific names

---
## Join GEVES with TAXREF-LD on scientific name, only on the GEVES species matched with FCU

In [14]:
# SPARQL query to TAXREF-LD to match a single GEVES scientific name (placeholder $geves_scientific_name)
queryTpl = Template(prefixes + '''
select distinct ("$geves_scientific_name" as ?geves_scientific_name) ?taxref_full_name ?taxref_name_type ?taxon ?rank 
from <http://taxref.mnhn.fr/lod/graph/classes/15.0>
from <http://taxref.mnhn.fr/lod/graph/vernacular/15.0>
from <http://taxref.mnhn.fr/lod/graph/concepts>
where {
    ?name
       a                      skos:Concept, <http://rs.tdwg.org/ontology/voc/TaxonName#TaxonName>;
       rdfs:label             ?taxref_full_name.

    { ?name taxrefp:isReferenceNameOf ?taxon. bind("pref" as ?taxref_name_type) }
    union
    { ?name taxrefp:isSynonymOf       ?taxon. bind("alt" as ?taxref_name_type) }

    ?taxon
       taxrefp:hasRank        ?rank.

    # All ranks up to spcecies but not above
    filter (?rank in (
        taxrefrk:Species,  taxrefrk:SemiSpecies, taxrefrk:MicroSpecies, taxrefrk:SubSpecies, taxrefrk:Natio, 
        taxrefrk:Varietas, taxrefrk:SubVarietas, taxrefrk:Forma,        taxrefrk:SubForma,   taxrefrk:FormaSpecies,
        taxrefrk:Linea,    taxrefrk:Clone,       taxrefrk:Race,         taxrefrk:Cultivar,   taxrefrk:Morpha,
        taxrefrk:Abberatio ))

    # TAXREF names have the authority and date, whereas GEVES names do not have the date.
    # => match the GEVES name as a subpart of the TAXREF name
    bind(str(lcase(?taxref_full_name)) as ?taxref_scn_sl)
    filter(strstarts(?taxref_scn_sl, "$geves_scientific_name"))
}
''')

In [17]:
# Set a max number of queries to submit. 0 = unlimited.
MAX_QUERIES = 0

# Result DataFrame
df_geves_taxref = pd.DataFrame()

idx = 1
df_fcu_geves_matched = df_fcu_geves.dropna()
unique_names = df_fcu_geves_matched.geves_scientific_name.unique()
for geves_scientific_name in unique_names:
    query = queryTpl.substitute(geves_scientific_name = geves_scientific_name.strip().lower())
    #print(query)
    
    print(f"---- Running query {idx}/{len(unique_names)} - geves_scientific_name = {geves_scientific_name}")
    %time _df = exec_sparql(taxref_endpoint, query)
    print(f'Number of results: {_df.shape[0]}')
    df_geves_taxref = df_geves_taxref.append(_df)
    
    # Keep track of GEVES names not matched with TAXREF
    if _df.shape[0] == 0:
        nomatch_row = {'taxref_full_name': None, 'taxref_name_type': None, 'taxon': None, 'geves_scientific_name': geves_scientific_name.strip().lower()}
        df_geves_taxref = df_geves_taxref.append(nomatch_row, ignore_index=True)
    
    idx = idx + 1
    if MAX_QUERIES > 0 and idx > MAX_QUERIES:
        break

---- Running query 1/85 - geves_scientific_name = prunus armeniaca l.
Wall time: 8 s
Number of results: 1
---- Running query 2/85 - geves_scientific_name = allium sativum l.
Wall time: 7.6 s
Number of results: 1
---- Running query 3/85 - geves_scientific_name = prunus amygdalus bartock
Wall time: 10.2 s
Number of results: 0
---- Running query 4/85 - geves_scientific_name = cynara scolymus
Wall time: 8.08 s
Number of results: 3
---- Running query 5/85 - geves_scientific_name = cynara cardunculus l.
Wall time: 9.19 s
Number of results: 1
---- Running query 6/85 - geves_scientific_name = asparagus officinalis l.
Wall time: 9.73 s
Number of results: 1
---- Running query 7/85 - geves_scientific_name = solanum melongena l.
Wall time: 9.77 s
Number of results: 1
---- Running query 8/85 - geves_scientific_name = avena sativa l.
Wall time: 8.19 s
Number of results: 1
---- Running query 9/85 - geves_scientific_name = avena nuda l.
Wall time: 8.86 s
Number of results: 1
---- Running query 10/85 -

In [20]:
#df_geves_taxref.to_csv("result2_geves_taxref.csv", index=False)
df_geves_taxref.to_excel("result2_geves_taxref.xlsx")
#df_geves_taxref = pd.read_csv("result2_geves_taxref.csv")

In [21]:
dataframe_preview(df_geves_taxref)

== Number of lines: 87
== Number of unique values:
geves_scientific_name    85
taxref_full_name         59
taxref_name_type          2
taxon                    57
rank                      2
dtype: int64


,geves_scientific_name,taxref_full_name,taxref_name_type,taxon,rank
0,prunus armeniaca l.,"Prunus armeniaca L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116041,http://taxref.mnhn.fr/lod/taxrank/Species
1,allium sativum l.,"Allium sativum L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/81505,http://taxref.mnhn.fr/lod/taxrank/Species
2,prunus amygdalus bartock,None,None,None,NaN
3,cynara scolymus,"Cynara scolymus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/93795,http://taxref.mnhn.fr/lod/taxrank/Species
4,cynara scolymus,Cynara scolymus subsp. cardunculus (L.) Bonnie...,alt,http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
5,cynara scolymus,"Cynara scolymus var. redonensis N.H.F.Desp., 1838",alt,http://taxref.mnhn.fr/lod/taxon/93795,http://taxref.mnhn.fr/lod/taxrank/Species
6,cynara cardunculus l.,"Cynara cardunculus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
7,asparagus officinalis l.,"Asparagus officinalis L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/84279,http://taxref.mnhn.fr/lod/taxrank/Species
8,solanum melongena l.,"Solanum melongena L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/124075,http://taxref.mnhn.fr/lod/taxrank/Species
9,avena sativa l.,"Avena sativa L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/85357,http://taxref.mnhn.fr/lod/taxrank/Species


#### Count only matches

In [22]:
dataframe_preview(df_geves_taxref.dropna())

== Number of lines: 59
== Number of unique values:
geves_scientific_name    57
taxref_full_name         59
taxref_name_type          2
taxon                    57
rank                      2
dtype: int64


,geves_scientific_name,taxref_full_name,taxref_name_type,taxon,rank
0,prunus armeniaca l.,"Prunus armeniaca L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116041,http://taxref.mnhn.fr/lod/taxrank/Species
1,allium sativum l.,"Allium sativum L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/81505,http://taxref.mnhn.fr/lod/taxrank/Species
3,cynara scolymus,"Cynara scolymus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/93795,http://taxref.mnhn.fr/lod/taxrank/Species
4,cynara scolymus,Cynara scolymus subsp. cardunculus (L.) Bonnie...,alt,http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
5,cynara scolymus,"Cynara scolymus var. redonensis N.H.F.Desp., 1838",alt,http://taxref.mnhn.fr/lod/taxon/93795,http://taxref.mnhn.fr/lod/taxrank/Species
6,cynara cardunculus l.,"Cynara cardunculus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
7,asparagus officinalis l.,"Asparagus officinalis L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/84279,http://taxref.mnhn.fr/lod/taxrank/Species
8,solanum melongena l.,"Solanum melongena L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/124075,http://taxref.mnhn.fr/lod/taxrank/Species
9,avena sativa l.,"Avena sativa L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/85357,http://taxref.mnhn.fr/lod/taxrank/Species
10,avena nuda l.,"Avena nuda L., 1756",pref,http://taxref.mnhn.fr/lod/taxon/85323,http://taxref.mnhn.fr/lod/taxrank/Species


### Matches:
- GEVES: 57 unique scientific names
- TAXREF-LD: 57 unique taxa

=> 32 GEVES names not matched with TAXREF-LD

---
## Join intermediate results: FCU-GEVES and GEVES-TAXREFLD

In [37]:
df_merge = pd.merge(df_fcu_geves, df_geves_taxref, on="geves_scientific_name", how='left')
df_merge.drop_duplicates(inplace=True)
dataframe_preview(df_merge)

== Number of lines: 1523
== Number of unique values:
fcu_concept               446
fcu_name                 1481
fcu_name_type               2
geves_lab_spe_dus          89
geves_scientific_name      85
taxref_full_name           59
taxref_name_type            2
taxon                      57
rank                        2
dtype: int64


,fcu_concept,fcu_name,fcu_name_type,geves_lab_spe_dus,geves_scientific_name,taxref_full_name,taxref_name_type,taxon,rank
0,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier,pref,Abricotier,prunus armeniaca l.,"Prunus armeniaca L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116041,http://taxref.mnhn.fr/lod/taxrank/Species
1,http://ontology.inrae.fr/frenchcropusage/Abric...,abricot,alt,None,None,NaN,NaN,NaN,NaN
2,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier pays,pref,None,None,NaN,NaN,NaN,NaN
3,http://ontology.inrae.fr/frenchcropusage/Abric...,abricot pays,alt,None,None,NaN,NaN,NaN,NaN
4,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier des antilles,alt,None,None,NaN,NaN,NaN,NaN
5,http://ontology.inrae.fr/frenchcropusage/Abric...,mamey,alt,None,None,NaN,NaN,NaN,NaN
6,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier-pays,alt,None,None,NaN,NaN,NaN,NaN
7,http://ontology.inrae.fr/frenchcropusage/Actin...,actinidia,pref,None,None,NaN,NaN,NaN,NaN
8,http://ontology.inrae.fr/frenchcropusage/Actin...,groseille de chine,alt,None,None,NaN,NaN,NaN,NaN
9,http://ontology.inrae.fr/frenchcropusage/Actin...,kiwi,alt,None,None,NaN,NaN,NaN,NaN


### Count only matches

In [31]:
dataframe_preview(df_merge.dropna())

== Number of lines: 70
== Number of unique values:
fcu_concept              64
fcu_name                 63
fcu_name_type             2
geves_lab_spe_dus        63
geves_scientific_name    57
taxref_full_name         59
taxref_name_type          2
taxon                    57
rank                      2
dtype: int64


,fcu_concept,fcu_name,fcu_name_type,geves_lab_spe_dus,geves_scientific_name,taxref_full_name,taxref_name_type,taxon,rank
0,http://ontology.inrae.fr/frenchcropusage/Abric...,abricotier,pref,Abricotier,prunus armeniaca l.,"Prunus armeniaca L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/116041,http://taxref.mnhn.fr/lod/taxrank/Species
13,http://ontology.inrae.fr/frenchcropusage/Ails,ail,pref,Ail,allium sativum l.,"Allium sativum L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/81505,http://taxref.mnhn.fr/lod/taxrank/Species
61,http://ontology.inrae.fr/frenchcropusage/Artic...,artichaut,pref,Artichaut,cynara scolymus,"Cynara scolymus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/93795,http://taxref.mnhn.fr/lod/taxrank/Species
62,http://ontology.inrae.fr/frenchcropusage/Artic...,artichaut,pref,Artichaut,cynara scolymus,Cynara scolymus subsp. cardunculus (L.) Bonnie...,alt,http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
63,http://ontology.inrae.fr/frenchcropusage/Artic...,artichaut,pref,Artichaut,cynara scolymus,"Cynara scolymus var. redonensis N.H.F.Desp., 1838",alt,http://taxref.mnhn.fr/lod/taxon/93795,http://taxref.mnhn.fr/lod/taxrank/Species
64,http://ontology.inrae.fr/frenchcropusage/Artic...,artichaut,pref,Artichaut,cynara cardunculus l.,"Cynara cardunculus L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
66,http://ontology.inrae.fr/frenchcropusage/Asperges,asperge,pref,Asperge,asparagus officinalis l.,"Asparagus officinalis L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/84279,http://taxref.mnhn.fr/lod/taxrank/Species
75,http://ontology.inrae.fr/frenchcropusage/Auber...,aubergine,pref,Aubergine,solanum melongena l.,"Solanum melongena L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/124075,http://taxref.mnhn.fr/lod/taxrank/Species
81,http://ontology.inrae.fr/frenchcropusage/Avoin...,avoine d'hiver,pref,Avoine d'hiver,avena sativa l.,"Avena sativa L., 1753",pref,http://taxref.mnhn.fr/lod/taxon/85357,http://taxref.mnhn.fr/lod/taxrank/Species
85,http://ontology.inrae.fr/frenchcropusage/Avoin...,avoine nue d'hiver,pref,Avoine nue d'hiver,avena nuda l.,"Avena nuda L., 1756",pref,http://taxref.mnhn.fr/lod/taxon/85323,http://taxref.mnhn.fr/lod/taxrank/Species


### Matches
- FCU: 64 unique concepts, 63 unique labels
- TAXREF-LD: 57 unique taxa from 2 ranks

### Exports

In [32]:
df_merge.to_excel("result3_fcu_geves_taxref.xlsx")

In [38]:
# Reshape the DataFrame for later merging with other methods
df_merge.dropna(inplace=True)
df_merge.drop(columns=['fcu_name_type', 'fcu_name', 'geves_scientific_name', 'taxref_name_type'], inplace=True)
df_merge.rename(columns={'taxref_full_name': 'taxref_ref_full_name'}, inplace=True)
df_merge.insert(1, 'method', 'geves')
df_merge.rename(columns={"geves_lab_spe_dus": "geves_name"}, inplace=True)
df_merge.insert(3, 'eppo_scientific_name', '')

In [39]:
dataframe_preview(df_merge)
df_merge.to_excel("result3_fcu_geves_taxref_merge.xlsx", index=False)
df_merge.to_csv("result3_fcu_geves_taxref_merge.csv", index=False)

== Number of lines: 70
== Number of unique values:
fcu_concept             64
method                   1
geves_name              63
eppo_scientific_name     1
taxref_ref_full_name    59
taxon                   57
rank                     2
dtype: int64


,fcu_concept,method,geves_name,eppo_scientific_name,taxref_ref_full_name,taxon,rank
0,http://ontology.inrae.fr/frenchcropusage/Abric...,geves,Abricotier,,"Prunus armeniaca L., 1753",http://taxref.mnhn.fr/lod/taxon/116041,http://taxref.mnhn.fr/lod/taxrank/Species
13,http://ontology.inrae.fr/frenchcropusage/Ails,geves,Ail,,"Allium sativum L., 1753",http://taxref.mnhn.fr/lod/taxon/81505,http://taxref.mnhn.fr/lod/taxrank/Species
61,http://ontology.inrae.fr/frenchcropusage/Artic...,geves,Artichaut,,"Cynara scolymus L., 1753",http://taxref.mnhn.fr/lod/taxon/93795,http://taxref.mnhn.fr/lod/taxrank/Species
62,http://ontology.inrae.fr/frenchcropusage/Artic...,geves,Artichaut,,Cynara scolymus subsp. cardunculus (L.) Bonnie...,http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
63,http://ontology.inrae.fr/frenchcropusage/Artic...,geves,Artichaut,,"Cynara scolymus var. redonensis N.H.F.Desp., 1838",http://taxref.mnhn.fr/lod/taxon/93795,http://taxref.mnhn.fr/lod/taxrank/Species
64,http://ontology.inrae.fr/frenchcropusage/Artic...,geves,Artichaut,,"Cynara cardunculus L., 1753",http://taxref.mnhn.fr/lod/taxon/93783,http://taxref.mnhn.fr/lod/taxrank/Species
66,http://ontology.inrae.fr/frenchcropusage/Asperges,geves,Asperge,,"Asparagus officinalis L., 1753",http://taxref.mnhn.fr/lod/taxon/84279,http://taxref.mnhn.fr/lod/taxrank/Species
75,http://ontology.inrae.fr/frenchcropusage/Auber...,geves,Aubergine,,"Solanum melongena L., 1753",http://taxref.mnhn.fr/lod/taxon/124075,http://taxref.mnhn.fr/lod/taxrank/Species
81,http://ontology.inrae.fr/frenchcropusage/Avoin...,geves,Avoine d'hiver,,"Avena sativa L., 1753",http://taxref.mnhn.fr/lod/taxon/85357,http://taxref.mnhn.fr/lod/taxrank/Species
85,http://ontology.inrae.fr/frenchcropusage/Avoin...,geves,Avoine nue d'hiver,,"Avena nuda L., 1756",http://taxref.mnhn.fr/lod/taxon/85323,http://taxref.mnhn.fr/lod/taxrank/Species
